In [31]:
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import time

In [32]:
# Premier League 2024-2025 Season 
url = "https://fbref.com/en/squads/822bd0ba/2024-2025/matchlogs/c9/schedule/Liverpool-Scores-and-Fixtures-Premier-League"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [33]:
response

<Response [200]>

In [34]:
# Matches already played in the PL 24-25 Season
match_report_tags = soup.find_all('td', {'data-stat': 'match_report'})
match_report_urls = []

for i in match_report_tags:
    if i.text == 'Match Report':
        match_report_urls.append("https://fbref.com/" + i.find('a')['href'])

df_list = []
for match_report_url in match_report_urls:
    df_list.append(extract_shooting_stats(match_report_url))

https://fbref.com//en/matches/a1d0d529/Ipswich-Town-Liverpool-August-17-2024-Premier-League
https://fbref.com//en/matches/09b1742e/Liverpool-Brentford-August-25-2024-Premier-League
https://fbref.com//en/matches/a7ab7a12/North-West-Derby-Manchester-United-Liverpool-September-1-2024-Premier-League
https://fbref.com//en/matches/674bfe9e/Liverpool-Nottingham-Forest-September-14-2024-Premier-League
https://fbref.com//en/matches/32a9539b/Liverpool-Bournemouth-September-21-2024-Premier-League
https://fbref.com//en/matches/f2633f1d/Wolverhampton-Wanderers-Liverpool-September-28-2024-Premier-League
https://fbref.com//en/matches/49ea224b/Crystal-Palace-Liverpool-October-5-2024-Premier-League
https://fbref.com//en/matches/99b4737c/Liverpool-Chelsea-October-20-2024-Premier-League
https://fbref.com//en/matches/68aa1099/Arsenal-Liverpool-October-27-2024-Premier-League
https://fbref.com//en/matches/7d114c70/Liverpool-Brighton-and-Hove-Albion-November-2-2024-Premier-League
https://fbref.com//en/matche

In [35]:
pd.concat(df_list).to_csv('data/df.csv',index=False)

In [30]:
def extract_shooting_stats(match_report_url):
    print(match_report_url)
    time.sleep(10)  # Adds a 2-second delay before making a request
    
    response = requests.get(match_report_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    match_title = soup.find('h1').text
    shots_table = soup.find('table', {'id': 'shots_all'})
    
    match_list = []
    player_list = []
    xg_list = []
    psxg_list = []
    outcome_list = []
    distance_list = []
    body_part_list = []
    
    for i in shots_table.find_all('tr'):
        player_element = i.find('td', {'data-stat': 'player'})

        if player_element:
            if player_element.text == 'Mohamed Salah' or player_element.text == 'Mohamed Salah (pen)':
                match_list.append(match_title)
                player_list.append(player_element.text)
                xg_list.append(i.find('td', {'data-stat':'xg_shot'}).text)
                psxg_list.append(i.find('td', {'data-stat':'psxg_shot'}).text)
                outcome_list.append(i.find('td', {'data-stat':'outcome'}).text)
                distance_list.append(i.find('td', {'data-stat':'distance'}).text)
                body_part_list.append(i.find('td', {'data-stat':'body_part'}).text)
    
    data = {'match': match_list, 'player': player_list, 'xG': xg_list, 'PSxG': psxg_list, 'outcome': outcome_list, 'distance': distance_list, 'body_part': body_part_list}
    
    df = pd.DataFrame(data)

    return df